In [ ]:
import nltk
# http://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype
import konlpy

#nltk.download('gutenberg')
#nltk.download('maxent_treebank_pos_tagger')
#nltk.download('stopwords')

In [ ]:
from konlpy.corpus import kobill    
files_ko = kobill.fileids()    
# C:\Anaconda3\envs\spark\Lib\site-packages\konlpy\data\corpus\kobill
doc_ko = kobill.open('위대한개츠비.txt').read()
doc_ko

### tokenize

In [ ]:
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk1.8.0_121\jre"

from konlpy.tag import Twitter
t = Twitter()

tokens_ko = t.morphs(doc_ko)
ko = nltk.Text(tokens_ko) 

print(len(ko.tokens))     
print(len(set(ko.tokens)))  
ko.vocab()  

In [ ]:
ko.count('개츠비') 

In [ ]:
ko.similar('개츠비')

In [ ]:
ko.collocations()

### 형태소 분석

In [ ]:
tags_ko = t.pos("아버지가방에들어가신다")
tags_ko

# topic modeling

In [ ]:
docs_ko = [kobill.open(i).read() for i in kobill.fileids()]
pos = lambda d: ['/'.join(p) for p in t.pos(d, stem=True, norm=True)]
texts_ko = [pos(doc) for doc in docs_ko]
print(texts_ko[0])

In [ ]:
from gensim import corpora
dictionary_ko = corpora.Dictionary(texts_ko)
dictionary_ko.save('ko.dict')

### TF-IDF

In [ ]:
from gensim import models
tf_ko = [dictionary_ko.doc2bow(text) for text in texts_ko]
tfidf_model_ko = models.TfidfModel(tf_ko)
tfidf_ko = tfidf_model_ko[tf_ko]
corpora.MmCorpus.serialize('ko.mm', tfidf_ko) 

# print first 10 elements of first document's tf-idf vector
print(tfidf_ko.corpus[0][:10])
# print top 10 elements of first document's tf-idf vector
first_top10 = sorted(tfidf_ko.corpus[0], key=lambda x: x[1], reverse=True)[:10]
print(first_top10)

In [ ]:
for w, v in first_top10:
    print(dictionary_ko.get(w))

### topic modeling 

In [ ]:
### LSI
ntopics, nwords = 3, 5
lsi_ko = models.lsimodel.LsiModel(tfidf_ko, id2word=dictionary_ko, num_topics=ntopics)
print(lsi_ko.print_topics(num_topics=ntopics, num_words=nwords))

In [ ]:
### LDA
import numpy as np; np.random.seed(42)  # optional
lda_ko = models.ldamodel.LdaModel(tfidf_ko, id2word=dictionary_ko, num_topics=ntopics)
print(lda_ko.print_topics(num_topics=ntopics, num_words=nwords))

In [ ]:
### HDP
import numpy as np; np.random.seed(42)  # optional
hdp_ko = models.hdpmodel.HdpModel(tfidf_ko, id2word=dictionary_ko)
print(hdp_ko.print_topics(num_topics=ntopics, num_words=nwords))